In [1]:
import os
from params import PARAMS
import pickle
import numpy as np
import json

In [2]:
import pandas as pd

def eval_runs(model_names, dataset_names, random_seeds):
    final_results = []
    for model_name in model_names:
        for dataset_name in dataset_names:
            subject_idc = json.load(open(os.path.join('./dataset/train', dataset_name + '.json'), 'r'))
            accs = []
            precs_macro = []
            precs_weighted = []
            recalls_macro = []
            recalls_weighted = []
            aurocs = []
            n_trainable_params = []

            for random_seed in random_seeds:
                path = os.path.join('output', model_name, dataset_name.rstrip('.pickle').lstrip('./dataset/train/'), str(random_seed), 'results.pickle')
                results = pickle.load(open(path, 'rb'))

                accs.append(results['cr']['accuracy'])
                precs_macro.append(results['cr']['macro avg']['precision'])
                precs_weighted.append(results['cr']['weighted avg']['precision'])
                recalls_macro.append(results['cr']['macro avg']['recall'])
                recalls_weighted.append(results['cr']['weighted avg']['recall'])
                aurocs.append(results['auroc'])
                n_trainable_params.append(results['n_params'])
            result_df = pd.DataFrame({'model_name': [model_name for i in random_seeds], 'dataset_name': [dataset_name for i in random_seeds],'train_idc': [subject_idc['train_idc'] for i in random_seeds], 'test_idc': [subject_idc['test_idc'] for i in random_seeds], 'random_seed': random_seeds, 'accuracy': accs, 'precision_macro': precs_macro, 'precision_weighted': precs_weighted, 'recall_macro': recalls_macro, 'recall_weighted': recalls_weighted, 'AUROC': aurocs, 'n_params': n_trainable_params})
            final_results.append(result_df)
    final_results = pd.concat(final_results).reset_index(drop=True)
    final_results.to_csv('./output/results.csv', index=False)

    std_per_dataset = final_results.groupby(['model_name', 'dataset_name']).std().drop(columns=['random_seed'])
    std_per_dataset = std_per_dataset.rename(columns={'accuracy': 'accuracy_std',
                        'precision_macro': 'precision_macro_std', 
                        'precision_weighted': 'precision_weighted_std', 
                        'recall_macro': 'recall_macro_std', 
                        'recall_weighted': 'recall_weighted_std'})
    std_per_dataset = std_per_dataset.drop(columns=['AUROC', 'n_params'])
    mean_per_dataset = final_results.groupby(['model_name', 'dataset_name']).mean().drop(columns='random_seed')
    results_per_dataset = pd.concat([mean_per_dataset, std_per_dataset], axis=1)
    results_per_dataset.to_csv('./output/results_per_dataset.csv')

    std_per_model = final_results.groupby(['model_name']).std().drop(columns=['random_seed'])
    std_per_model = std_per_model.rename(columns={'accuracy': 'accuracy_std',
                        'precision_macro': 'precision_macro_std', 
                        'precision_weighted': 'precision_weighted_std', 
                        'recall_macro': 'recall_macro_std', 
                        'recall_weighted': 'recall_weighted_std'})
    std_per_model = std_per_model.drop(columns=['AUROC', 'n_params'])
    mean_per_model = final_results.groupby(['model_name']).mean().drop(columns='random_seed')
    results_per_model = pd.concat([mean_per_model, std_per_model], axis=1)
    results_per_model.to_csv('./output/results_per_model.csv')
    print(results_per_model)

In [4]:
model_names = ['imagine_fcn', 'imagine_cnn', 'imagine_rnn', 'imagine_gcn', 'imagine_gcn_auto']
# model_names = ['imagine_gcn_auto']
random_seeds = PARAMS['RANDOM_SEEDS']
print(random_seeds)
dataset_names = [f'cross_subject_data_{i}_5_subjects' for i in range(5)]
final_results = eval_runs(model_names, dataset_names, random_seeds[:1])
# for k, v in final_results.items():
#     print(k)
#     print(v['accuracy'])

[688933, 890203, 517391, 195645, 705223]
                  accuracy  precision_macro  precision_weighted  recall_macro  \
model_name                                                                      
imagine_cnn       0.302778         0.303401            0.303965      0.302958   
imagine_fcn       0.244097         0.145516            0.142928      0.254628   
imagine_gcn       0.240972         0.131827            0.133634      0.248990   
imagine_gcn_auto  0.311111         0.312034            0.313130      0.311561   
imagine_rnn       0.264583         0.264662            0.265535      0.265109   

                  recall_weighted     AUROC  n_params  accuracy_std  \
model_name                                                            
imagine_cnn              0.302778  0.535190  371220.0      0.045182   
imagine_fcn              0.244097  0.503101  354308.0      0.024200   
imagine_gcn              0.240972  0.499310  354308.0      0.014179   
imagine_gcn_auto         0.311111  0

In [5]:
final_results = pd.read_csv('./output/results.csv')

std_per_dataset = final_results.groupby(['model_name', 'dataset_name']).std().drop(columns=['random_seed'])
std_per_dataset = std_per_dataset.rename(columns={'accuracy': 'accuracy_std',
                    'precision_macro': 'precision_macro_std', 
                    'precision_weighted': 'precision_weighted_std', 
                    'recall_macro': 'recall_macro_std', 
                    'recall_weighted': 'recall_weighted_std'})
std_per_dataset = std_per_dataset.drop(columns=['AUROC', 'n_params'])
mean_per_dataset = final_results.groupby(['model_name', 'dataset_name']).mean().drop(columns='random_seed')
results_per_dataset = pd.concat([mean_per_dataset, std_per_dataset], axis=1)
results_per_dataset.reset_index()

,model_name,dataset_name,accuracy,precision_macro,precision_weighted,recall_macro,recall_weighted,AUROC,n_params,accuracy_std,precision_macro_std,precision_weighted_std,recall_macro_std,recall_weighted_std
0,imagine_cnn,cross_subject_data_0_5_subjects,0.375000,0.377748,0.379809,0.376771,0.375000,0.584442,371220.0,NaN,NaN,NaN,NaN,NaN
1,imagine_cnn,cross_subject_data_1_5_subjects,0.262153,0.263011,0.262727,0.262549,0.262153,0.508265,371220.0,NaN,NaN,NaN,NaN,NaN
2,imagine_cnn,cross_subject_data_2_5_subjects,0.315972,0.314224,0.314700,0.312782,0.315972,0.541784,371220.0,NaN,NaN,NaN,NaN,NaN
3,imagine_cnn,cross_subject_data_3_5_subjects,0.288194,0.288426,0.289856,0.288250,0.288194,0.525631,371220.0,NaN,NaN,NaN,NaN,NaN
4,imagine_cnn,cross_subject_data_4_5_subjects,0.272569,0.273596,0.272733,0.274438,0.272569,0.515830,371220.0,NaN,NaN,NaN,NaN,NaN
5,imagine_fcn,cross_subject_data_0_5_subjects,0.218750,0.054688,0.047852,0.250000,0.218750,0.500000,354308.0,NaN,NaN,NaN,NaN,NaN
6,imagine_fcn,cross_subject_data_1_5_subjects,0.248264,0.252365,0.253794,0.250897,0.248264,0.500643,354308.0,NaN,NaN,NaN,NaN,NaN
7,imagine_fcn,cross_subject_data_2_5_subjects,0.237847,0.059462,0.056571,0.250000,0.237847,0.500000,354308.0,NaN,NaN,NaN,NaN,NaN
8,imagine_fcn,cross_subject_data_3_5_subjects,0.282986,0.302906,0.302304,0.272244,0.282986,0.514864,354308.0,NaN,NaN,NaN,NaN,NaN
9,imagine_fcn,cross_subject_data_4_5_subjects,0.232639,0.058160,0.054121,0.250000,0.232639,0.500000,354308.0,NaN,NaN,NaN,NaN,NaN


In [6]:
std_per_model = final_results.groupby(['model_name']).std().drop(columns=['random_seed'])
std_per_model = std_per_model.rename(columns={'accuracy': 'accuracy_std',
                    'precision_macro': 'precision_macro_std', 
                    'precision_weighted': 'precision_weighted_std', 
                    'recall_macro': 'recall_macro_std', 
                    'recall_weighted': 'recall_weighted_std'})
std_per_model = std_per_model.drop(columns=['AUROC', 'n_params'])
mean_per_model = final_results.groupby(['model_name']).mean().drop(columns='random_seed')
results_per_model = pd.concat([mean_per_model, std_per_model], axis=1)
results_per_model

,accuracy,precision_macro,precision_weighted,recall_macro,recall_weighted,AUROC,n_params,accuracy_std,precision_macro_std,precision_weighted_std,recall_macro_std,recall_weighted_std
model_name,,,,,,,,,,,,
imagine_cnn,0.302778,0.303401,0.303965,0.302958,0.302778,0.535190,371220.0,0.045182,0.045794,0.046751,0.045297,0.045182
imagine_fcn,0.244097,0.145516,0.142928,0.254628,0.244097,0.503101,354308.0,0.024200,0.121937,0.124575,0.009855,0.024200
imagine_gcn,0.240972,0.131827,0.133634,0.248990,0.240972,0.499310,354308.0,0.014179,0.095581,0.098726,0.002284,0.014179
imagine_gcn_auto,0.311111,0.312034,0.313130,0.311561,0.311111,0.541029,358404.0,0.032186,0.032500,0.033356,0.031777,0.032186
imagine_rnn,0.264583,0.264662,0.265535,0.265109,0.264583,0.510070,357556.0,0.028171,0.025810,0.024578,0.027655,0.028171


In [ ]:
path = 'output/imagine_cnn/cross_subject_data_0/195645/results.pickle'
results = pickle.load(open(path, 'rb'))

In [ ]:
results['history']['val_acc']

[0.28912869095802307,
 0.2906447947025299,
 0.30393293499946594,
 0.29135823249816895,
 0.28877195715904236,
 0.29626327753067017,
 0.2916257977485657,
 0.294568806886673,
 0.2949255406856537,
 0.30009809136390686,
 0.2949255406856537,
 0.29474717378616333,
 0.2933202385902405,
 0.30214929580688477,
 0.29082316160202026,
 0.28029966354370117,
 0.29296353459358215,
 0.2868099510669708,
 0.29465797543525696,
 0.29421207308769226,
 0.2916257977485657,
 0.2868991196155548,
 0.2931418716907501,
 0.29055559635162354,
 0.2914474308490753,
 0.28377774357795715,
 0.2889503240585327,
 0.29055559635162354,
 0.28841522336006165,
 0.29251760244369507,
 0.29180416464805603,
 0.2860964834690094,
 0.28663158416748047,
 0.2899313271045685,
 0.2874342203140259,
 0.2857397496700287,
 0.29171496629714966,
 0.28288593888282776,
 0.29091233015060425,
 0.28636404871940613,
 0.2914474308490753,
 0.2894854247570038,
 0.2901988625526428,
 0.2857397496700287,
 0.2838669419288635,
 0.28618568181991577,
 0.2915365